# Multi-Vendor Network Automation with NAPALM

## Part 4 - Make Configuration Changes with NAPALM



Everything we've done thus far is retrieve information from our network device. While this is a very powerful tool to have, we'll eventually need to make changes on our device. Does NAPALM support this?

The good news is - yes, it totally does. However, this part isn't always as abstracted as the "getter" functions we've seen thus far. By retrieving data, we can focus only on very particular use cases, and NAPALM will do the hard work of translating the retrieved data from vendor-specific APIs into a common format. The problem with trying to do this with configuration data is that vendors have wildly different configuration syntaxes, and it's not very useful to only focus on configuring a small subset of a device's capabilities.

> Hopefully this will eventually improve, with the advent of [OpenConfig](http://www.openconfig.net/), models which can be used with the [napalm-yang](https://napalm-yang.readthedocs.io/en/latest/root/supported_models/index.html) project to provide vendor-agnostic configurations. Unfortunately only a handful of vendors currently support OpenConfig.

So, what we end up having to do is construct a vendor-specific configuration outside of NAPALM, such as with a templating language like Jinja, and then pass this in to one of the configuration functions in NAPALM. Let's do that now.

Let's assume you have a variable titled `vqfx1_config` that contains a partial configuration for setting the description of an interface. Ideally you would have built this from a template using Jinja and something like YAML, but we'll just create this explicitly for now so we can learn how to apply it using NAPALM:

> Try changing the description yourself in the XML structure below, by editing the text between the `<description>` and `</description>` tags.

In [3]:
vqfx1_config = """
<configuration>
    <interfaces>
        <interface>
            <name>em0</name>
            <unit>
                <name>0</name>
                <description>This is em0, and it connects to something.</description>
            </unit>
        </interface>
    </interfaces>
</configuration>
"""

Again, we need to recreate the NAPALM driver and device objects so we can use them in this notebook:

In [4]:
import napalm
driver = napalm.get_network_driver("junos")
device = driver(hostname="vqfx1", username="antidote", password="antidotepassword")
device.open()

Next, we can use a function we haven't seen yet, called `load_merge_candidate`, and pass our configuration string as the `config` parameter.

> If we had the config stored in a file, we could use the `filename` parameter instead.

The `merge` portion of that function name is the "strategy" used for applying that config. In short, this will attempt to merge the existing configuration with the changes you're proposing, and only change what needs to change to incorporate the new configuration.

In [17]:
device.load_merge_candidate(config=vqfx1_config)

As is common on modern network operating systems, the changes we've proposed have gone into a candidate configuration store, which means they've been sent to the device, but the device hasn't started using the new changes. We have to `commit` them in order to do that. As is custom in these situations, we can use the `compare_config` to see the exact diff of what will happen to the configuration if we were to commit it now:

In [18]:
print(device.compare_config())


If we didn't like the diff we saw above, we could call `device.discard_config()` to get rid of the changes we proposed as candidate. However, this looks good to us, so we can apply the changes with a `commit_config`:

In [19]:
device.commit_config()

> The function `load_replace_candidate` is similar, but instead of attempting to merge the configuration, it overwrites the entire configuration with what you pass in. **USE WITH CAUTION** as you will need to make sure your configuration is exactly what it needs to be.

Now that our change is applied, we can use the `get_interfaces` function to see the description applied to the operational state of the device:

In [24]:
device.get_interfaces()['em0.0']['description']

Often, we make mistakes and need to roll back a change. Let's say we made a typo in the description and just want to undo what we just committed. No worries - the `rollback` function does this for us.

In [25]:
device.rollback()
device.get_interfaces()['em0.0']['description']

NAPALM also comes with a few [built-in templates] you can use to perform some common tasks without building your own configuration.

> You can see that even the small number of templates included in the project include examples that will only work on some vendor devices. This is the challenge with trying to unify existing configuration paradigms.

That's it for now! In future versions of the lesson (or perhaps in other lessons) we'll dive deeper into some of the more specific tools within NAPALM for targeted workflows. For now, check out the [NAPALM documentation](https://napalm.readthedocs.io) for more information.
